In [ ]:
from pathlib import Path

import pandas as pd

from analytics.app.data.load import list_pipelines
from analytics.app.data.transform import dfs_models_and_evals, logs_dataframe

%load_ext autoreload
%autoreload 2

In [ ]:
pipelines_dir = Path("/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/arxiv/30_performance")

In [ ]:
pipelines = list_pipelines(pipelines_dir)
max_pipeline_id = max(pipelines.keys())
pipelines

In [ ]:
from analytics.app.data.load import load_pipeline_logs

pipeline_logs = {p_id: load_pipeline_logs(p_id, pipelines_dir) for (p_id, (_, p_path)) in pipelines.items()}

In [ ]:
# mode:
pipeline_id = 762  # performancetrigger_num_misclass-10000-exp-0.6-red-False--int20000

# doesn't do anything unless include_composite_model = True
composite_model_variant = "currently_active_model"

patch_yearbook = True
dataset_id = "arxiv_kaggle_test"
eval_handler = "periodic-current"
metric = "Accuracy"
include_composite_model = False

# Wrangle data

In [ ]:
pipeline_log = pipeline_logs[pipeline_id]
pipeline_ref = f"{pipeline_id}".zfill(len(str(max_pipeline_id))) + f" - {pipelines[pipeline_id][0]}"

df_all = logs_dataframe(pipeline_log, pipeline_ref)

df_logs_models, _, df_eval_single = dfs_models_and_evals(
    # subtracting would interfere with yearbook patching
    pipeline_log,
    df_all["sample_time"].max(),
    pipeline_ref,
)

df_adjusted = df_eval_single


df_adjusted = df_adjusted[
    (df_adjusted["dataset_id"] == dataset_id)
    & (df_adjusted["eval_handler"] == eval_handler)
    & (df_adjusted["metric"] == metric)
]

# in percent (0-100)
df_adjusted["value"] = df_adjusted["value"] * 100

In [ ]:
df_adjusted = df_adjusted.sort_values(by=["interval_center"])

In [ ]:
# Add composite model

assert df_adjusted["pipeline_ref"].nunique() <= 1
# add the pipeline time series which is the performance of different models stitched together dep.
# w.r.t which model was active
pipeline_composite_model = df_adjusted[df_adjusted[composite_model_variant]]
pipeline_composite_model["model_idx"] = 0
pipeline_composite_model["id_model"] = 0

label_map = {k: f"{k}" for k, v in df_adjusted[["model_idx", "id_model"]].values}
label_map[0] = "Pipeline composite model"

if include_composite_model:
    df_adjusted = pd.concat([pipeline_composite_model, df_adjusted])
else:
    df_adjusted["model_idx"] = df_adjusted["model_idx"]

# Create Plot

In [ ]:
df_adjusted = df_adjusted.sort_values(by=["interval_center"])
df_adjusted["interval_center"] = df_adjusted["interval_center"].dt.to_period("M")
df_adjusted

In [ ]:
df_train_end_years_per_model = df_logs_models[["model_idx", "real_train_end"]]
df_train_end_years_per_model["real_train_end"] = df_train_end_years_per_model["real_train_end"].dt.to_period("M")
df_train_end_years_per_model

In [ ]:
df_merged = df_adjusted.merge(df_train_end_years_per_model, on="model_idx", how="left")
df_merged

In [ ]:
df_merged.groupby(["real_train_end", "interval_center"]).size()

In [ ]:
# build heatmap matrix dataframe:
df_merged["real_train_end"] = df_merged["real_train_end"].apply(lambda x: pd.Period(x, freq="M"))
heatmap_data = df_merged.pivot(index=["real_train_end"], columns="interval_center", values="value")

In [ ]:
heatmap_data.index.min(), heatmap_data.index.max()

In [ ]:
heatmap_data.index

In [ ]:
from analytics.plotting.common.heatmap import build_heatmap
from analytics.plotting.common.save import save_plot

fig = build_heatmap(
    heatmap_data,
    reverse_col=True,
    x_custom_ticks=[
        (i, f"{period.to_timestamp().strftime('%b %Y')}")
        for i, period in list(enumerate(heatmap_data.columns))[::1]
        if period in [pd.Period("Mar 2000"), pd.Period("Mar 2009"), pd.Period("Mar 2020")]
    ],
    y_custom_ticks=[
        (i, f"{period.to_timestamp().strftime('%b %Y')}".replace(" ", "\n"))
        for i, period in list(enumerate(heatmap_data.index))[::1]
        if period in [pd.Period("Jun 2000"), pd.Period("Jun 2009"), pd.Period("May 2020")]
    ],
    y_label="Trained up to",
    x_label="Evaluation Year",
    title_label="Arxiv PerformanceTrigger\nExp. Acc=60% | NumMiscl=10k | No Reduction",
    color_label="Accuracy %",
    width_factor=1,
    height_factor=0.7,
    # grid_alpha=0.4,
    grid_alpha=0.0,
    # disable_horizontal_grid=True,
    # cbar=False,
    df_logs_models=df_logs_models,
    x_axis="period",
)
save_plot(fig, "arxiv_trigger_heatmap_performance_single_dynamic")